<hr />

# Baseball App Example

In this example we use Pandas, sklearn, and Bokeh to explore the Lahman Baseball Statistics database.



In [1]:
## import statements
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
import bokeh
from bokeh.models import ColumnDataSource
#from bokeh.io import output_notebook, show
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
sns.set(color_codes=True)
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5, "figure.figsize": (20, 12)})
label_size = 24
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = label_size 
mpl.rcParams['axes.labelsize'] = label_size 

## Read data and explore

In [4]:
import sqlite3
con = sqlite3.connect("../data/baseball/lahman2013.sqlite")

with con:
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    data = cur.fetchall()
print([datum[0] for datum in data])

['AllstarFull', 'Appearances', 'AwardsManagers', 'AwardsPlayers', 'AwardsShareManagers', 'AwardsSharePlayers', 'Batting', 'BattingPost', 'Fielding', 'FieldingOF', 'FieldingPost', 'HallOfFame', 'Managers', 'ManagersHalf', 'Master', 'Pitching', 'PitchingPost', 'Salaries', 'Schools', 'SchoolsPlayers', 'SeriesPost', 'Teams', 'TeamsFranchises', 'TeamsHalf', 'temp']


In [5]:
# Read sqlite query results into a pandas DataFrame
with con:
    df_salaries = pd.read_sql_query("SELECT * from Salaries", con)

# verify that result of SQL query is stored in the dataframe
print(df_salaries.head())

   yearID teamID lgID   playerID    salary
0    1985    ATL   NL  barkele01  870000.0
1    1985    ATL   NL  bedrost01  550000.0
2    1985    ATL   NL  benedbr01  545000.0
3    1985    ATL   NL   campri01  633333.0
4    1985    ATL   NL  ceronri01  625000.0


In [6]:
list(df_salaries.teamID.unique())

['ATL',
 'BAL',
 'BOS',
 'CAL',
 'CHA',
 'CHN',
 'CIN',
 'CLE',
 'DET',
 'HOU',
 'KCA',
 'LAN',
 'MIN',
 'ML4',
 'MON',
 'NYA',
 'NYN',
 'OAK',
 'PHI',
 'PIT',
 'SDN',
 'SEA',
 'SFN',
 'SLN',
 'TEX',
 'TOR',
 'COL',
 'FLO',
 'ANA',
 'ARI',
 'MIL',
 'TBA',
 'LAA',
 'WAS',
 'MIA']

In [7]:
gb = df_salaries.groupby('teamID')
salary_by_team = gb['salary'].agg([np.mean, np.std, np.max, np.min] )
salary_by_team['ratio'] = salary_by_team.amax / salary_by_team.amin

In [8]:
salary_by_team.head()

,mean,std,amax,amin,ratio
teamID,,,,,
ANA,1.895109e+06,2.667147e+06,13166667.0,150000.0,87.777780
ARI,2.428196e+06,2.971520e+06,16000000.0,170000.0,94.117647
ATL,2.130475e+06,3.373518e+06,16061802.0,60000.0,267.696700
BAL,1.785712e+06,2.491109e+06,17000000.0,60000.0,283.333333
BOS,2.692114e+06,3.861802e+06,22500000.0,60000.0,375.000000


In [9]:
salary_by_team.index

Index(['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CAL', 'CHA', 'CHN', 'CIN', 'CLE',
       'COL', 'DET', 'FLO', 'HOU', 'KCA', 'LAA', 'LAN', 'MIA', 'MIL', 'MIN',
       'ML4', 'MON', 'NYA', 'NYN', 'OAK', 'PHI', 'PIT', 'SDN', 'SEA', 'SFN',
       'SLN', 'TBA', 'TEX', 'TOR', 'WAS'],
      dtype='object', name='teamID')

In [10]:
salary_by_team['teamID'] = salary_by_team.index

In [11]:
df = salary_by_team.sort_values('mean')
source = ColumnDataSource(df)
p = bokeh.plotting.figure(x_range=list(df.index))
p.scatter(x="teamID", y="mean", source=source)
bokeh.io.show(p)

Hmm, can't read the y axis very well...

In [12]:
df = df.sort_values('mean')
source = ColumnDataSource(df)
p = bokeh.plotting.figure(x_range=list(df["teamID"]))
p.scatter(x="teamID", y="mean", source=source)
p.xaxis.major_label_orientation = np.pi/3

bokeh.io.show(p)

Let's view a max versus ratio

In [13]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,lasso_select"

df.sort_values('mean', inplace=True)
source = ColumnDataSource(df)
s1 = bokeh.plotting.figure(title="Pay Avg",x_range=source.data["teamID"], tools=TOOLS, width=500)
s1.scatter(x="teamID", y="mean", source=source)
s1.xaxis.major_label_orientation = np.pi/3

s2 = bokeh.plotting.figure(title="Pay Ratio", x_range=s1.x_range, tools=TOOLS, width=500)
s2.scatter(x="teamID", y="ratio", source=source)
s2.xaxis.major_label_orientation = np.pi/3

p = bokeh.plotting.gridplot([[s1, s2]])
bokeh.io.show(p)

<hr/>

Now let's join on the AllStars table to see how max salaries and all star count correlate.

In [14]:
with con:
    df_allstar = pd.read_sql_query("SELECT * from AllstarFull", con)

In [15]:
df_allstar.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,aaronha01,1955,0,NLS195507120,ML1,NL,1.0,NaN
1,aaronha01,1956,0,ALS195607100,ML1,NL,1.0,NaN
2,aaronha01,1957,0,NLS195707090,ML1,NL,1.0,9.0
3,aaronha01,1958,0,ALS195807080,ML1,NL,1.0,9.0
4,aaronha01,1959,1,NLS195907070,ML1,NL,1.0,9.0


In [16]:
df_allstar_count = df_allstar.groupby('teamID')['playerID'].agg(np.count_nonzero)

In [17]:
df_allstar_count.head()

teamID
ANA     14
ARI     33
ARL      1
ATL    134
BAL    169
Name: playerID, dtype: int64

In [18]:
df_allstar_count[df_allstar_count == df_allstar_count.max()]

teamID
NYA    412
Name: playerID, dtype: int64

Now let's use this as the size of the circles in the scatter plot

In [19]:
df_allstar_count_circle_size = 10 * df_allstar_count / df_allstar_count.max() + 10

<hr/>

Now lets join the data to all_star sizes

In [20]:
df = pd.DataFrame()

In [21]:
df = df.append(salary_by_team)

In [22]:
salary_by_team['all_stars'] = df_allstar_count_circle_size

In [23]:
df.append?

In [24]:
df.head()

,mean,std,amax,amin,ratio,teamID
teamID,,,,,,
ANA,1.895109e+06,2.667147e+06,13166667.0,150000.0,87.777780,ANA
ARI,2.428196e+06,2.971520e+06,16000000.0,170000.0,94.117647,ARI
ATL,2.130475e+06,3.373518e+06,16061802.0,60000.0,267.696700,ATL
BAL,1.785712e+06,2.491109e+06,17000000.0,60000.0,283.333333,BAL
BOS,2.692114e+06,3.861802e+06,22500000.0,60000.0,375.000000,BOS


In [25]:
df = salary_by_team.sort_values("amax")
print(df.head())
source = ColumnDataSource(df)
p = bokeh.plotting.figure(x_range=list(df["teamID"]))
p.scatter(x="teamID", y="amax", size="all_stars", source=source, fill_alpha=0.5, )
p.xaxis.major_label_orientation = np.pi/3

bokeh.io.show(p)

                mean           std        amax      amin       ratio teamID  \
teamID                                                                        
CAL     7.390732e+05  9.606285e+05   5375000.0   60000.0   89.583333    CAL   
ML4     6.132436e+05  8.535337e+05   5875000.0   60000.0   97.916667    ML4   
TBA     1.528400e+06  2.011091e+06  10125000.0  170000.0   59.558824    TBA   
MON     7.074589e+05  1.192678e+06  11500000.0   60000.0  191.666667    MON   
KCA     1.299026e+06  1.951653e+06  13000000.0   60000.0  216.666667    KCA   

        all_stars  
teamID             
CAL     11.626214  
ML4     11.165049  
TBA     10.703883  
MON     11.699029  
KCA     12.014563  


<hr/>

Now let's make this an interactive plot!

In [26]:
# Read sqlite query results into a pandas DataFrame
with con:
    df_salaries = pd.read_sql_query("SELECT yearID, teamID, salary from Salaries", con)
    df_allstar = pd.read_sql_query("SELECT yearID, teamID, playerID from AllstarFull", con)

def compute_df(year=2012):
    salaries_for_year = df_salaries[ df_salaries.yearID == year]
    max_salaries = salaries_for_year.groupby('teamID')['salary'].agg([np.max])
    max_salaries.sort_values('amax', ascending=False, inplace=True)
    allstar_for_year = df_allstar[ df_allstar.yearID == year]
    allstar_count = allstar_for_year.groupby('teamID')['playerID'].count()
    allstar_count_circle_size = 10 * allstar_count / allstar_count.max() + 10
    result = pd.DataFrame(max_salaries)
    result['all_stars']= allstar_count_circle_size
    result['teamID'] = result.index
    return result

In [27]:
compute_df().head()

,amax,all_stars,teamID
teamID,,,
NYA,30000000.0,14.444444,NYA
LAA,24187500.0,12.222222,LAA
NYN,23145011.0,13.333333,NYN
MIN,23000000.0,11.111111,MIN
DET,23000000.0,15.555556,DET


In [28]:
source = ColumnDataSource(compute_df())
p = bokeh.plotting.figure(x_range=list(source.data["teamID"]))
p.scatter(x="teamID", y="amax", size="all_stars", source=source, fill_alpha=0.5, )
p.xaxis.major_label_orientation = np.pi/3

In [29]:
from ipywidgets import interact, widgets 
def update(year):
    df = compute_df(year)
    source.data['all_stars'] = df['all_stars']
    source.data['amax'] = df['amax']
    bokeh.io.push_notebook()

In [30]:
bokeh.io.show(p)

In [31]:
interact(update, year=widgets.IntSlider(min=1985, max=2013, value=2013))

<function __main__.update>

In [32]:
with con:
    df_pitching = pd.read_sql_query("SELECT * from Pitching", con)


In [33]:
df_pitching.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')